In [16]:
import pytest
from matsciml.datasets import lips
from matsciml.datasets.lips import LiPSDataset, lips_devset
from matsciml.datasets import transforms
import pytorch_lightning as pl
from matsciml.models.pyg import EGNN
from matsciml.models.base import ScalarRegressionTask, MaceEnergyForceTask
from matsciml.lightning.data_utils import MatSciMLDataModule
from matsciml.datasets.transforms import DistancesTransform
from matsciml.models.pyg.mace.modules.blocks import *
from matsciml.models.pyg.mace.modules.models import ScaleShiftMACE
import matsciml
import e3nn

In [11]:
from matsciml.datasets.transforms import PointCloudToGraphTransform
# SchNet uses RBFs, and expects edge features corresponding to atom-atom distances
dm = MatSciMLDataModule.from_devset(
    "LiPSDataset",dset_kwargs={"transforms":[PointCloudToGraphTransform("pyg", cutoff_dist=5.0)]}
)
# run a quick training loop
# trainer = pl.Trainer(fast_dev_run=1000)
# trainer.fit(task, datamodule=dm)

In [12]:
from matsciml.models.pyg.mace import data, modules, tools
from matsciml.models.pyg.mace.modules.utils import compute_mean_std_atomic_inter_energy
from matsciml.models.pyg.mace.tools import (
    AtomicNumberTable,
    atomic_numbers_to_indices,
    to_one_hot,
    torch_geometric,
    voigt_to_matrix,
)

In [13]:
def to_numpy(t: torch.Tensor) -> np.ndarray:
    return t.cpu().detach().numpy()

def compute_mean_std_atomic_inter_energy(
    data_loader: torch.utils.data.DataLoader,
    atomic_energies: np.ndarray,
) -> Tuple[float, float]:
    atomic_energies_fn = AtomicEnergiesBlock(atomic_energies=atomic_energies)

    avg_atom_inter_es_list = []

    for batch in data_loader:

        graph = batch.get("graph")
        atomic_numbers: torch.Tensor = getattr(graph, "atomic_numbers")
        z_table=tools.get_atomic_number_table_from_zs(atomic_numbers.numpy())

        indices = atomic_numbers_to_indices(atomic_numbers, z_table=z_table)
        node_attrs = to_one_hot(
            torch.tensor(indices, dtype=torch.long).unsqueeze(-1),
            num_classes=len(z_table))
        node_e0 = atomic_energies_fn(node_attrs)
        graph_e0s = scatter_sum(
            src=node_e0, index=graph.batch, dim=-1, dim_size=graph.num_graphs
        )
        graph_sizes = graph.ptr[1:] - graph.ptr[:-1]
        avg_atom_inter_es_list.append(
            (batch['energy'] - graph_e0s) / graph_sizes
        )  # {[n_graphs], }

    avg_atom_inter_es = torch.cat(avg_atom_inter_es_list)  # [total_n_graphs]
    mean = to_numpy(torch.mean(avg_atom_inter_es)).item()
    std = to_numpy(torch.std(avg_atom_inter_es)).item()

    return mean, std

In [14]:
dm.setup()
Train_loader=dm.train_dataloader()
dataset_iter = iter(Train_loader)
batch=next(dataset_iter)
atomic_energies=np.array([-13.663181292231226, -1029.2809654211628, -2042.0330099956639])
compute_mean_std_atomic_inter_energy(Train_loader,atomic_energies)

(1231.470703125, 0.006572697777301073)

In [15]:
model_config = dict(
        r_max=5.0,
        num_bessel=8,
        num_polynomial_cutoff=5,
        max_ell=3,
        interaction_cls= RealAgnosticResidualInteractionBlock ,
        num_interactions=5,
        num_elements=3,
        hidden_irreps=e3nn.o3.Irreps('16x0e+16x1o+16x2e'),
        atomic_energies=torch.Tensor([-13.663181292231226, -1029.2809654211628, -2042.0330099956639]),
        avg_num_neighbors=14.38,
        atomic_numbers=(1,6,8),
        correlation=3,
        gate=torch.nn.functional.silu,
        interaction_cls_first=RealAgnosticResidualInteractionBlock,
        MLP_irreps=e3nn.o3.Irreps('16x0e'),
        atomic_inter_scale=0.006573571357876062,
        atomic_inter_shift=1231.4705810546875
    )

In [17]:
# task = ScalarRegressionTask(
#     encoder_class=ScaleShiftMACE,
#     # kwargs to be passed into the creation of SchNet model
#     encoder_kwargs=model_config,
#     # which keys to use as targets
#     task_keys=["energy"],
#     output_kwargs={
#       "block_type": "IrrepOutputBlock",
#       "input_dim": "0e", 
#       "hidden_dim": "30x0e + 10x1e", 
#       "output_dim": "0e",
#       "activation": ["torch.nn.SiLU", None],
#       "residual":False
#     }
# )

task = ScalarRegressionTask(
    encoder_class=ScaleShiftMACE,
    # kwargs to be passed into the creation of SchNet model
    encoder_kwargs=model_config,
    # which keys to use as targets
    task_keys=["energy","force"],
    output_kwargs={
      "block_type": "IdentityOutputBlock",
      "output_dim": 1,
      "hidden_dim": None

      }
)

c:\Users\DIKSHIKA\miniconda3\envs\matsciml\lib\site-packages\torch\jit\_check.py:172: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
f:\Vaibhav\M3RG\matsciml\matsciml\models\pyg\mace\modules\blocks.py:131: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(atomic_energies, dtype=torch.get_default_dtype()),


TypeError: __init__() missing 1 required positional argument: 'output_dim'

In [8]:
task

ScalarRegressionTask(
  (encoder): ScaleShiftMACE(
    (atom_embedding): Embedding(100, 16, padding_idx=0)
    (node_embedding): LinearNodeEmbeddingBlock(
      (linear): Linear(3x0e -> 16x0e | 48 weights)
    )
    (radial_embedding): RadialEmbeddingBlock(
      (bessel_fn): BesselBasis(r_max=5.0, num_basis=8, trainable=False)
      (cutoff_fn): PolynomialCutoff(p=5.0, r_max=5.0)
    )
    (spherical_harmonics): SphericalHarmonics()
    (atomic_energies_fn): AtomicEnergiesBlock(energies=[-13.6632, -1029.2810, -2042.0330])
    (interactions): ModuleList(
      (0): RealAgnosticResidualInteractionBlock(
        (linear_up): Linear(16x0e -> 16x0e | 256 weights)
        (conv_tp): TensorProduct(16x0e x 1x0e+1x1o+1x2e+1x3o -> 16x0e+16x1o+16x2e+16x3o | 64 paths | 64 weights)
        (conv_tp_weights): FullyConnectedNet[8, 64, 64, 64, 64]
        (linear): Linear(16x0e+16x1o+16x2e+16x3o -> 16x0e+16x1o+16x2e+16x3o | 1024 weights)
        (skip_tp): FullyConnectedTensorProduct(16x0e x 3x0e -> 

In [9]:
trainer = pl.Trainer(max_epochs=100)
trainer.fit(task, datamodule=dm)

GPU available: True (cuda), used: False


TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
c:\Users\DIKSHIKA\miniconda3\envs\matsciml\lib\site-packages\pytorch_lightning\trainer\setup.py:175: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(

  | Name         | Type           | Params
------------------------------------------------
0 | encoder      | ScaleShiftMACE | 184 K 
1 | loss_func    | MSELoss        | 0     
2 | output_heads | ModuleDict     | 0     
------------------------------------------------
184 K     Trainable params
0         Non-trainable params
184 K     Total params
0.739     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

c:\Users\DIKSHIKA\miniconda3\envs\matsciml\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]True



RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [13]:
from matsciml.models.pyg.mace.modules.models import ScaleShiftMACE1
model=ScaleShiftMACE(**model_config)

In [14]:
dm.setup()
Train_loader=dm.train_dataloader()
dataset_iter = iter(Train_loader)
batch=next(dataset_iter)

In [15]:
batch['graph'].edge_index

tensor([[  4,   6,   6,  ..., 663, 663, 663],
        [  3,   1,   2,  ..., 655, 660, 661]])

In [16]:
out=model(batch)

In [ ]:
batch['graph']

In [ ]:
task(batch)

In [ ]:
inputs=model.read_batch(batch)

In [ ]:
outputs=model._forward(**inputs)

In [ ]:
out

In [ ]:
outputs